In [ ]:
# task2_change_point_model.py

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymc3 as pm
import arviz as az
import seaborn as sns

# --------------------------
# 1. Load Brent Oil Data
# --------------------------
df = pd.read_csv("data/brent_oil_prices.csv", parse_dates=["Date"])
df.sort_values("Date", inplace=True)
df.set_index("Date", inplace=True)

# --------------------------
# 2. Compute Log Returns
# --------------------------
df["log_return"] = np.log(df["Price"] / df["Price"].shift(1))
df.dropna(inplace=True)

returns = df["log_return"].values
n = len(returns)

# --------------------------
# 3. Plot Raw Data & Returns
# --------------------------
plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
sns.lineplot(x=df.index, y=df["Price"])
plt.title("Brent Oil Price")

plt.subplot(1, 2, 2)
sns.lineplot(x=df.index, y=df["log_return"])
plt.title("Log Returns")
plt.tight_layout()
plt.show()

# --------------------------
# 4. Bayesian Change Point Model with PyMC3
# --------------------------
with pm.Model() as model:
    tau = pm.DiscreteUniform("tau", lower=0, upper=n)

    mu_1 = pm.Normal("mu_1", mu=0, sigma=0.05)
    mu_2 = pm.Normal("mu_2", mu=0, sigma=0.05)

    sigma_1 = pm.HalfNormal("sigma_1", sigma=0.02)
    sigma_2 = pm.HalfNormal("sigma_2", sigma=0.02)

    mu = pm.math.switch(tau >= np.arange(n), mu_1, mu_2)
    sigma = pm.math.switch(tau >= np.arange(n), sigma_1, sigma_2)

    obs = pm.Normal("obs", mu=mu, sigma=sigma, observed=returns)

    trace = pm.sample(2000, tune=1000, target_accept=0.95, return_inferencedata=True)

# --------------------------
# 5. Diagnostic Summary
# --------------------------
az.plot_trace(trace, var_names=["tau", "mu_1", "mu_2"])
plt.tight_layout()
plt.show()

summary = az.summary(trace, var_names=["tau", "mu_1", "mu_2", "sigma_1", "sigma_2"])
print(summary)

# --------------------------
# 6. Identify Change Point
# --------------------------
tau_samples = trace.posterior["tau"].values.flatten()
tau_median = int(np.median(tau_samples))
change_date = df.index[tau_median]
print(f"Most probable change point: {change_date.date()}")

mu_1_mean = summary.loc["mu_1", "mean"]
mu_2_mean = summary.loc["mu_2", "mean"]
pct_change = ((mu_2_mean - mu_1_mean) / abs(mu_1_mean)) * 100

print(f"Mean log return before: {mu_1_mean:.5f}")
print(f"Mean log return after: {mu_2_mean:.5f}")
print(f"Relative change in return: {pct_change:.2f}%")

# --------------------------
# 7. Visualize Change Point
# --------------------------
plt.figure(figsize=(10, 5))
plt.plot(df.index, df["log_return"], label="Log Returns")
plt.axvline(change_date, color="red", linestyle="--", label=f"Change Point: {change_date.date()}")
plt.title("Brent Oil Log Returns with Bayesian Change Point")
plt.xlabel("Date")
plt.ylabel("Log Return")
plt.legend()
plt.tight_layout()
plt.show()

# --------------------------
# 8. Match Change Point to Event
# --------------------------
events = pd.read_csv("data/oil_market_events.csv", parse_dates=["Start_Date"])
events["Delta_Days"] = abs((events["Start_Date"] - change_date).dt.days)
closest_event = events.loc[events["Delta_Days"].idxmin()]

print("\n📌 Closest Event to Change Point:")
print(closest_event)

